# Analyzing Data Using SQL

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), containing statistics about all of the countries on Earth. The Factbook contains [demographic information](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db) like the following:

-  population — the global population.
-  population_growth — the annual population growth rate, as a percentage.
-  area — the total land and water area.

## Table of Contents
---
- [Connecting to the database](#Connecting-to-the-Database)
- [Overview of the Data](#Overview-of-the-Data)
- [Summary Statistics](#Summary-Statistics)
- [Densely Populated Countries](#Densely-Populated-Countries)
- [Sparsely Populated Countries](#Sparsely-Populated-Countries)
- [Populated Islands](#Populated-Islands)
- [Exploring Population Density](#Exploring-Population-Density)
- [Conclusion](#Conclusion)

## Connecting to the Database

First we connect our Jupyter Notebook to our database file:

In [4]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

##  Overview of the Data

In [5]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [6]:
%%sql
SELECT *
FROM facts
LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


The facts table comprises 11 columns with the following titles:
>- **id** —  Entry row number.
>- **code** — The country's [internet code](https://www.cia.gov/the-world-factbook/field/internet-country-code/).
>- **name** — Name of the country.
>- **area** — The country's total area (both land and water).
>- **area_land** — The country's land area in square kilometers.
>- **area_water** — The country's water area in square kilometers.
>- **population** —  Whole number of people or inhabitants in the country.
>- **population_growth** — The country's population growth as a percentage.
>- **birth_rate** — The number of births per year per 1,000 inhabitants.
>- **death_rate** — The number of deaths per year per 1,000 inhabitants.
>- **migration_rate** — The difference between the number of persons entering and leaving the country during the year per 1,000 persons. 
  
We will then check the total number of records in the facts table:

In [16]:
%%sql
-- Calculate the total number of records
SELECT COUNT(*) AS 'Total Records',
       COUNT(DISTINCT name) AS Countries
FROM facts;

Done.


Total Records,Countries
261,261


We know there are only 195 countries in the world in 2023. 

As of July 2011, The World Factbook comprises 267 entities divided into the following categories:
>- **Independent countries:** The CIA defines these as people "politically organized into a sovereign state with a definite territory." There are 195 entities in this category.
>- **Others:** Places set apart from the list of independent countries. Currently, there are two: Taiwan and the European Union.
>- **Dependencies and Areas of Special Sovereignty:** Places affiliated with another country. There are 58 in total.
>- **Miscellaneous:** Antarctica and places in dispute. There are six such entities.
>- **Other entities:** The world and the oceans. There are five oceans and the world (the World entry is intended as a summary of the other entries).

Additional information about this classification can be found in the [world factbook wikipedia page](https://en.wikipedia.org/wiki/The_World_Factbook#:~:text=Entities%20listed,-Main%20article%3A%20List&text=The%20CIA%20defines%20these%20as,category%2C%20there%20are%20195%20entities.&text=Places%20set%20apart%20from%20the,Taiwan%20and%20the%20European%20Union.).

We can now check for these entities in our data:

In [17]:
%%sql
-- check for 'non-country' data
SELECT *
FROM facts
WHERE name LIKE '%union' 
    OR name LIKE '%ocean'
    OR name LIKE '%island'
    OR name = 'World';

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
197,ee,European Union,4324782,None,None,513949445,0.25,10.2,10.2,2.5
199,kt,Christmas Island,135,135,0,1530,1.11,None,None,None
203,nf,Norfolk Island,36,36,0,2210,0.01,None,None,None
208,ip,Clipperton Island,6,6,0,None,None,None,None,None
222,bv,Bouvet Island,49,49,0,None,None,None,None,None
244,bq,Navassa Island,5,5,0,None,None,None,None,None
248,wq,Wake Island,6,6,0,None,None,None,None,None
256,xq,Arctic Ocean,None,None,None,None,None,None,None,None
257,zh,Atlantic Ocean,None,None,None,None,None,None,None,None
258,xo,Indian Ocean,None,None,None,None,None,None,None,None


## Summary Statistics

In [7]:
%%sql
-- Minimum and Maximum Population
SELECT MIN(population) AS 'Minimum population', MAX(population) AS 'Maximum population',
MIN(population_growth) AS 'Minimum population growth', MAX(population_growth) AS
'Maximum population growth'
FROM facts;


Done.


Minimum population,Maximum population,Minimum population growth,Maximum population growth
0,7256490011,0.0,4.02


## Investigating Outliers

In [8]:
%%sql
SELECT name, population
FROM facts
WHERE population = ( SELECT MAX(population) 'Maximum Population' FROM facts) or 
population = ( SELECT MIN(population) 'Minimum Population' FROM facts);


Done.


name,population
Antarctica,0
World,7256490011


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0.

In [9]:
%%sql
SELECT name, population
FROM facts
WHERE population = ( SELECT MIN(population)  FROM facts WHERE name IS NOT 'Antarctica') OR 
population = ( SELECT MAX(population) FROM facts WHERE name IS NOT 'World') ;


Done.


name,population
China,1367485388
Pitcairn Islands,48


The minimum population is 48, and the maximum is over 1.3 billion, We can now identify the highest and least population growth countries.


In [10]:
%%sql
SELECT name, population_growth
FROM facts
WHERE population_growth = ( SELECT MAX(population_growth) FROM facts WHERE name IS NOT 'World') or 
population_growth = ( SELECT MIN(population_growth)  FROM facts WHERE name IS NOT 'Antarctica');


Done.


name,population_growth
South Sudan,4.02
Holy See (Vatican City),0.0
Cocos (Keeling) Islands,0.0
Greenland,0.0
Pitcairn Islands,0.0


We can see that China has the most people. South Sudan has the highest growth rate of 4.02.

## Densely Populated Countries

We'll identify countries that have the following:
1. Above-average values for population.
2. Below-average values for area.

In [50]:
%%sql
SELECT AVG(population) 'Average Population',  AVG(area) 'Average Area' 
FROM facts 
WHERE name NOT LIKE '%union' 
    AND name NOT LIKE '%ocean'
    AND name NOT LIKE '%island'
    AND name NOT LIKE '%islands'
    AND name NOT LIKE 'Antarctica'
    AND name NOT LIKE 'World';


Done.


Average Population,Average Area
32103886.619469028,594908.6


The average population centers around 32 million inhabitants. 

In [51]:
%%sql 
SELECT name AS "5 Most Populated Countries", population,
ROUND(CAST(population AS Float) / CAST((SELECT MAX(population) FROM facts) AS Float) , 2) AS 'World Proportion'
FROM facts
WHERE population > 0 AND name NOT IN ('World', 'European Union')
ORDER BY 2 DESC
LIMIT 5;

Done.


5 Most Populated Countries,population,World Proportion
China,1367485388,0.19
India,1251695584,0.17
United States,321368864,0.04
Indonesia,255993674,0.04
Brazil,204259812,0.03


 China is the world's most populated country, with about 1.37 billion people. India is next with about 1.26 billion people. 

** Approximately 36%of the world's total population comes from China (18.8%) and India (17.2%) alone.**

The United States (4.4% of the world total), Indonesia (3.5%), and Brazil (2.8%) follow far behind and altogether the top 5 countries contribute **46%** to the total global population.

## Sparsely Populated Countries

In [58]:
%%sql
SELECT name, population, area
FROM facts
WHERE population < 594908
    AND name NOT LIKE '%union' 
    AND name NOT LIKE '%ocean'
    AND name NOT LIKE '% island%'
    AND name NOT LIKE '%islands'
    AND name NOT LIKE 'Antarctica'
ORDER BY population
LIMIT 15;

Done.


name,population,area
Holy See (Vatican City),842,0
Niue,1190,260
Tokelau,1337,12
Svalbard,1872,62045
Montserrat,5241,102
Saint Pierre and Miquelon,5657,242
Saint Barthelemy,7237,None
"Saint Helena, Ascension, and Tristan da Cunha",7795,308
Nauru,9540,21
Tuvalu,10869,26


## Populated Islands

In [68]:
%%sql
SELECT name, population, area
FROM facts
WHERE population < 594908 AND population != 'NONE'
    AND name NOT LIKE '%ocean'
    AND name NOT LIKE 'Antarctica'
    AND name LIKE '% island%' OR name LIKE '%islands%'
ORDER BY population DESC
LIMIT 15;

Done.


name,population,area
Solomon Islands,622469,28896
Virgin Islands,103574,1910
Marshall Islands,72191,181
Cayman Islands,56092,264
Northern Mariana Islands,52344,464
Turks and Caicos Islands,50280,948
Faroe Islands,50196,1393
British Virgin Islands,33454,151
Cook Islands,9838,236
Falkland Islands (Islas Malvinas),3361,12173


The least populated Countries are Holy See -Vatican City (842), Niue (1190), Tokelau	(1337), Svalbard (1872) and Montserrat (5241).
    
The least populated Islands are the Pitcairn Islands (48 inhabitants), Cocos Islands (596), Christmas Island (1530), Norfolk Island (2210) and Falkland Islands (Islas Malvinas) (3361)

## Exploring Population Density

Our data provides variables that we can use to estimate population density as follows:

$$ 
Population\ area = \frac{Total\ population}{Land\ area\ (km^2)}
$$


In [72]:
%%sql
SELECT name, area_land, population,
       CAST(CAST(population AS Float) / CAST(area_land AS Float) AS Integer) AS 'Population Density'
FROM facts
 WHERE name NOT IN (SELECT name FROM facts
                     WHERE population = 0
                         OR population IS NULL
                         OR area_land = 0
                         OR area_land IS NULL  )
ORDER BY 4 DESC
LIMIT 10;

Done.


name,area_land,population,Population Density
Macau,28,592731,21168
Monaco,2,30535,15267
Singapore,687,5674472,8259
Hong Kong,1073,7141106,6655
Gaza Strip,360,1869055,5191
Gibraltar,6,29258,4876
Bahrain,760,1346613,1771
Maldives,298,393253,1319
Malta,316,413965,1310
Bermuda,54,70196,1299


All of these locations have a very low land area. The top five have a combined land area of less than 3,000 km².  These areas are also highly urbanized centers of wealth and finance, with cities and skyscrapers.

We may be interested in a densely populated country where the population is greater than the average world population
and the area is greater than the average world area.

In [73]:
%%sql
SELECT name, population, area_land,
CAST(CAST(population AS Float) / CAST(area_land AS Float) AS Integer) AS 'Population Density'
FROM facts
WHERE area_land < 594908.6
AND population > 32103886
ORDER BY 4 DESC
 LIMIT 10;

Done.


name,population,area_land,Population Density
Bangladesh,168957745,130170,1297
"Korea, South",49115196,96920,506
Japan,126919659,364485,348
Philippines,100998376,298170,338
Vietnam,94348835,310070,304
United Kingdom,64088222,241930,264
Germany,80854408,348672,231
Italy,61855120,294140,210
Uganda,37101745,197100,188
Thailand,67976405,510890,133


Bangladesh has the highest population and low land area with a population density of 1,297 people per square km. Bangladesh's population density is more than two times the values recorded for South Korea. Japan, the Philippines, and Vietnam follow respectively.

In [77]:
%%sql
SELECT name, population, area_land,
ROUND(CAST(population AS Float) / CAST(area_land AS Float), 3) AS 'Population Density'
FROM facts
WHERE area_land > 594908.6
AND population < 32103886
ORDER BY 4
 LIMIT 10;

Done.


name,population,area_land,Population Density
Greenland,57733,2166086,0.027
Mongolia,2992908,1553556,1.926
Namibia,2212307,823290,2.687
Australia,22751014,7682300,2.961
Mauritania,3596702,1030700,3.49
Libya,6411776,1759540,3.644
Kazakhstan,18157122,2699700,6.726
Central African Republic,5391539,622984,8.654
Chad,11631456,1259200,9.237
Bolivia,10800882,1083301,9.97


 The least densely populated territory is Greenland, with a rate of 0.027 people per square km then Mongolia, Namibia, Australia, and Mauritania follow behind.


 ##  Conclusion

The CIA collected this data between 2014 and 2015. As a result, our findings are limited to the period the data was published. They may not reflect the current state of the world's countries.

Accounting for 46% of the world's total population, China, India, the United States, Indonesia, and Brazil are the most populated countries. The least populated world locations are majorly Islands and dependencies. 

Due to their small land area, Macau, Monaco, Singapore, and Hong Kong are more densely populated than the rest of the world. However, when considering larger land areas, Bangladesh, South Korea, and Japan are the most densely populated countries.
